**✨아래 셀의 epochs 300으로 변경하기**

**✨런타임 유지**  
f12키 - console 창에서 아래 코드 실행

```
function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click()
}
setInterval(KeepClicking,60000)
```
**✨모든 셀 실행**  

# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-167-g5deff14 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.4/78.2 GB disk)


# Modify detect.py
class
- 0: bicycle
- 1: motorcycle
- 2: scooter
- 3: with helmet
- 4: without helmet

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
#%cat /content/yolov5/detect.py

# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license
"""
Run YOLOv5 detection inference on images, videos, directories, globs, YouTube, webcam, streams, etc.

Usage - sources:
    $ python detect.py --weights yolov5s.pt --source 0                               # webcam
                                                     img.jpg                         # image
                                                     vid.mp4                         # video
                                                     screen                          # screenshot
                                                     path/                           # directory
                                                     list.txt                        # list of images
                                                     list.streams                    # list of streams
                                                     'path/*.jpg'                    # glob
                                                     'https:/

#아래가 최종 코드

In [ ]:
# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license
"""
Run YOLOv5 detection inference on images, videos, directories, globs, YouTube, webcam, streams, etc.

Usage - sources:
    $ python detect.py --weights yolov5s.pt --source 0                               # webcam
                                                     img.jpg                         # image
                                                     vid.mp4                         # video
                                                     screen                          # screenshot
                                                     path/                           # directory
                                                     list.txt                        # list of images
                                                     list.streams                    # list of streams
                                                     'path/*.jpg'                    # glob
                                                     'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                                                     'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream

Usage - formats:
    $ python detect.py --weights yolov5s.pt                 # PyTorch
                                 yolov5s.torchscript        # TorchScript
                                 yolov5s.onnx               # ONNX Runtime or OpenCV DNN with --dnn
                                 yolov5s_openvino_model     # OpenVINO
                                 yolov5s.engine             # TensorRT
                                 yolov5s.mlmodel            # CoreML (macOS-only)
                                 yolov5s_saved_model        # TensorFlow SavedModel
                                 yolov5s.pb                 # TensorFlow GraphDef
                                 yolov5s.tflite             # TensorFlow Lite
                                 yolov5s_edgetpu.tflite     # TensorFlow Edge TPU
                                 yolov5s_paddle_model       # PaddlePaddle
"""

import argparse
import os
import platform
import sys
from pathlib import Path

import torch

FILE = Path(__file__).resolve()
ROOT = FILE.parents[0]  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadScreenshots, LoadStreams
from utils.general import (LOGGER, Profile, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, smart_inference_mode


@smart_inference_mode()
def run(
        weights=ROOT / 'yolov5s.pt',  # model path or triton URL
        source=ROOT / 'data/images',  # file/dir/URL/glob/screen/0(webcam)
        data=ROOT / 'data/coco128.yaml',  # dataset.yaml path
        imgsz=(640, 640),  # inference size (height, width)
        conf_thres=0.25,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        project=ROOT / 'runs/detect',  # save results to project/name
        name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        dnn=False,  # use OpenCV DNN for ONNX inference
        vid_stride=1,  # video frame-rate stride
):
    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
    is_url = source.lower().startswith(('rtsp://', 'rtmp://', 'http://', 'https://'))
    webcam = source.isnumeric() or source.endswith('.streams') or (is_url and not is_file)
    screenshot = source.lower().startswith('screen')
    if is_url and is_file:
        source = check_file(source)  # download

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
    stride, names, pt = model.stride, model.names, model.pt
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Dataloader
    bs = 1  # batch_size
    if webcam:
        view_img = check_imshow(warn=True)
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
        bs = len(dataset)
    elif screenshot:
        dataset = LoadScreenshots(source, img_size=imgsz, stride=stride, auto=pt)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    model.warmup(imgsz=(1 if pt or model.triton else bs, 3, *imgsz))  # warmup
    seen, windows, dt = 0, [], (Profile(), Profile(), Profile())
    for path, im, im0s, vid_cap, s in dataset:
        with dt[0]:
            im = torch.from_numpy(im).to(model.device)
            im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
            im /= 255  # 0 - 255 to 0.0 - 1.0
            if len(im.shape) == 3:
                im = im[None]  # expand for batch dim

        # Inference
        with dt[1]:
            visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
            pred = model(im, augment=augment, visualize=visualize)

        # NMS
        with dt[2]:
            pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)

        # Second-stage classifier (optional)
        # pred = utils.general.apply_classifier(pred, classifier_model, im, im0s)

        # Process predictions
        for i, det in enumerate(pred):  # per image
            seen += 1
            if webcam:  # batch_size >= 1
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
                s += f'{i}: '
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
            s += '%gx%g ' % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()

                #modify
                # print('here det')
                # print(det)
                helmet_det = [] # new det list for 헬멧 객체
                nohelmet_det = [] # new det list for 헬멧 없는 사람 얼굴 객체
                person_det = [] # new det list for 사람 객체(withhelmet + withouthelmet)
                det_lst = det.tolist() # tensor를 list 형태로 변환
                # print(det_lst)
                for i in det_lst:
                  if i[5]==3: # 헬멧인 객체만 새 리스트(helmet_det)에 추가하기
                    helmet_det.append(i[:4])
                  if i[5]==4: # 헬멧 없는 사람 객체만 새 리스트(nohelmet_det)에 추가하기
                    nohelmet_det.append(i[:4])
                  if i[5]==3 or i[5]==4: # 사람 얼굴 객체(with + without helmet)만 새 리스트(person_det)에 추가하기
                    person_det.append(i[:4])

                # print(helmet_det)

                # Print results
                # modify
                total_bicycle = 0
                total_mtcycle = 0
                total_scooter = 0

                for c in det[:, 5].unique():
                    n = (det[:, 5] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
                    if c == 0:
                      total_bicycle = int(n) # 한 그림당 이륜차 종류 개수 구하기
                    if c == 1:
                      total_mtcycle = int(n)
                    if c == 2:
                      total_scooter = int(n)

                # Write results
                # modify 디텍트된 헬멧안쓴사람 개수(이륜차 종류별)
                cnt_nohmt_bicycle = 0
                cnt_nohmt_mtcycle = 0
                cnt_nohmt_scooter = 0

                # 이륜차에서 검출된 사람 수
                # cnt_person_bicycle = 0
                # cnt_person_mtcycle= 0
                cnt_person_scooter = 0
                scooter_ppl = False

                # modify - IoU
                def IoU(box1, box2):
                  # box = (x1, y1, x2, y2)
                  box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
                  box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

                  # obtain x1, y1, x2, y2 of the intersection
                  x1 = max(box1[0], box2[0])
                  y1 = max(box1[1], box2[1])
                  x2 = min(box1[2], box2[2])
                  y2 = min(box1[3], box2[3])

                  # compute the width and height of the intersection
                  w = max(0, x2 - x1 + 1)
                  h = max(0, y2 - y1 + 1)

                  inter = w * h
                  iou = inter / (box1_area + box2_area - inter)
                  return iou

                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        with open(f'{txt_path}.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        annotator.box_label(xyxy, label, color=colors(c, True))

                        #modify
                        # print("here new_xyxy")
                        if c not in [3,4]: # 검출된 객체가 이륜차일 때 (헬멧이 아닐 때)
                          new_xyxy = [(1.05*xyxy[0]-0.05*xyxy[2]),(1.7*xyxy[1]-0.7*xyxy[3]),(1.05*xyxy[2]-0.05*xyxy[0]),xyxy[3]] # 양옆 각 5%, 위 70% 증가된 바운딩박스
                          # 넓은 증가 바운딩박스 그리기
                          annotator.box_label(new_xyxy, color=colors(9, True)) # plots.py

                          # 각 텐서값을 숫자로 변환 - 하나의 리스트로 만들기
                          new_xyxy2 = [float(1.05*xyxy[0]-0.05*xyxy[2]),float(1.7*xyxy[1]-0.7*xyxy[3]),float(1.05*xyxy[2]-0.05*xyxy[0]),float(xyxy[3])] # 양옆 각 5%, 위 70% 증가된 바운딩박스

                          #한 바운딩박스에서 헬멧X 개수 검출
                          for nohelmet in nohelmet_det:
                            # print("helmet",nohelmet)
                            # print("newxyxy",new_xyxy2)
                            # print("IoU",IoU(nohelmet, new_xyxy2)) # 헬멧 객체와 지금 이륜차의 IoU값 구하기
                            if IoU(nohelmet, new_xyxy2) > 0: # 박스가 서로 겹치면 헬멧안쓴사람 존재로 판단
                              if int(c)==0:
                                cnt_nohmt_bicycle += 1
                              if int(c)==1:
                                cnt_nohmt_mtcycle += 1
                              if int(c)==2:
                                cnt_nohmt_scooter += 1


                          # 한 스쿠터(전동킥보드)에 2명 이상 탔을 위험 상황 검출
                          for person in person_det:
                            # print("helmet",helmet)
                            # print("newxyxy",new_xyxy2)
                            # print("IoU",IoU(helmet, new_xyxy2)) # 헬멧 객체와 지금 이륜차의 IoU값 구하기

                            if IoU(person, new_xyxy2) > 0: # 박스가 서로 겹치면 헬멧을 쓰고 있는 사람 존재로 판단
                              if int(c)==2:
                                cnt_person_scooter += 1

                          # 검출 결과 나타내기
                          if cnt_person_scooter >=2:
                            s += f"\n⚠️ {cnt_person_scooter}명 탑승한 전동킥보드 존재  "



                    if save_crop:
                        save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)


            # Stream results
            im0 = annotator.result()
            if view_img:
                if platform.system() == 'Linux' and p not in windows:
                    windows.append(p)
                    cv2.namedWindow(str(p), cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)  # allow window resize (Linux)
                    cv2.resizeWindow(str(p), im0.shape[1], im0.shape[0])
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path[i] != save_path:  # new video
                        vid_path[i] = save_path
                        if isinstance(vid_writer[i], cv2.VideoWriter):
                            vid_writer[i].release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                        save_path = str(Path(save_path).with_suffix('.mp4'))  # force *.mp4 suffix on results videos
                        vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer[i].write(im0)



            if cnt_nohmt_bicycle != 0:
              s += f"\n⚠️ 한 자전거에서 헬멧 미착용자 {cnt_nohmt_bicycle}명 존재  "
            if cnt_nohmt_mtcycle != 0:
              s += f"\n⚠️ 한 오토바이에서 헬멧 미착용자 {cnt_nohmt_mtcycle}명 존재  "
            if cnt_nohmt_scooter != 0:
              s += f"\n⚠️ 한 전동킥보드에서 헬멧 미착용자 {cnt_nohmt_scooter}명 존재  "

        # modify
        # wo_hmt_bicycle = (total_bicycle - cnt_hmt_bicycle) # without helmet 개수 (자전거에서)
        # wo_hmt_mtcycle = (total_mtcycle - cnt_hmt_mtcycle) # without helmet 개수 (오토바이에서)
        # wo_hmt_scooter = (total_scooter - cnt_hmt_scooter) # without helmet 개수 (스쿠터에서)

        # if wo_hmt_bicycle != 0:
        #   s += f"\n자전거 주행자 중 헬멧 미착용자 {wo_hmt_bicycle}명 존재 "
        # if wo_hmt_mtcycle != 0:
        #   s += f"\n오토바이 주행자 중 헬멧 미착용자 {wo_hmt_mtcycle}명 존재 "
        # if wo_hmt_scooter != 0:
        #   s += f"\n전동킥보드 주행자 중 헬멧 미착용자 {wo_hmt_scooter}명 존재 "

        # 스쿠터 2명 이상 탄 경우 검출
        # if scooter_ppl:
        #   s += f"\n한 스쿠터에 {cnt_person_scooter}명 탑승 "



        # Print time (inference-only)
        LOGGER.info(f"{s}{'' if len(det) else '(no detections), '}") #{dt[1].dt * 1E3:.1f}ms

    # Print results
    t = tuple(x.t / seen * 1E3 for x in dt)  # speeds per image
    LOGGER.info(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {(1, 3, *imgsz)}' % t)
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
    if update:
        strip_optimizer(weights[0])  # update model (to fix SourceChangeWarning)


def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', nargs='+', type=str, default=ROOT / 'yolov5s.pt', help='model path or triton URL')
    parser.add_argument('--source', type=str, default=ROOT / 'data/images', help='file/dir/URL/glob/screen/0(webcam)')
    parser.add_argument('--data', type=str, default=ROOT / 'data/coco128.yaml', help='(optional) dataset.yaml path')
    parser.add_argument('--imgsz', '--img', '--img-size', nargs='+', type=int, default=[640], help='inference size h,w')
    parser.add_argument('--conf-thres', type=float, default=0.25, help='confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.45, help='NMS IoU threshold')
    parser.add_argument('--max-det', type=int, default=1000, help='maximum detections per image')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--view-img', action='store_true', help='show results')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
    parser.add_argument('--save-crop', action='store_true', help='save cropped prediction boxes')
    parser.add_argument('--nosave', action='store_true', help='do not save images/videos')
    parser.add_argument('--classes', nargs='+', type=int, help='filter by class: --classes 0, or --classes 0 2 3')
    parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
    parser.add_argument('--augment', action='store_true', help='augmented inference')
    parser.add_argument('--visualize', action='store_true', help='visualize features')
    parser.add_argument('--update', action='store_true', help='update all models')
    parser.add_argument('--project', default=ROOT / 'runs/detect', help='save results to project/name')
    parser.add_argument('--name', default='exp', help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--line-thickness', default=3, type=int, help='bounding box thickness (pixels)')
    parser.add_argument('--hide-labels', default=False, action='store_true', help='hide labels')
    parser.add_argument('--hide-conf', default=False, action='store_true', help='hide confidences')
    parser.add_argument('--half', action='store_true', help='use FP16 half-precision inference')
    parser.add_argument('--dnn', action='store_true', help='use OpenCV DNN for ONNX inference')
    parser.add_argument('--vid-stride', type=int, default=1, help='video frame-rate stride')
    opt = parser.parse_args()
    opt.imgsz *= 2 if len(opt.imgsz) == 1 else 1  # expand
    print_args(vars(opt))
    return opt


def main(opt):
    check_requirements(exclude=('tensorboard', 'thop'))
    run(**vars(opt))


if __name__ == '__main__':
    opt = parse_opt()
    main(opt)


# Detect
det
first 4 means x, y
5th => accuracy
6th => class #



xyxy => 네모 박스 왼쪽 밑 점의 x,y좌표가 [0], [1], 네모 박스 오른쪽 위 점의 x,y좌표가 [2], [3] 번째임!

In [ ]:
%cd yolov5

/content/yolov5


In [ ]:
!python detect.py --weights /content/drive/MyDrive/YOLOhelmet/models/YOLO5m.pt --img 640 --conf 0.4 --source /content/drive/MyDrive/YOLOhelmet/images/test
# display.Image(filename='runs/detect/exp', width=600)

detect: weights=['/content/drive/MyDrive/YOLOhelmet/models/YOLO5m.pt'], source=/content/drive/MyDrive/YOLOhelmet/images/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-166-g54e9515 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
custom_YOLOv5m summary: 290 layers, 20869098 parameters, 0 gradients, 47.9 GFLOPs
image 1/4 /content/drive/MyDrive/YOLOhelmet/images/test/1.jpg: 448x640 1 Bicycle, 1 With Helmet, 1 Without Helmet, 
image 2/4 /content/drive/MyDrive/YOLOhelmet/images/test/2.png: 448x640 2 Bicycles, 1 With Helmet, 1 Without Helmet, 
⚠️ 한 자전거에서 헬멧 미착용자 1명 존재  
image 3/4 /content/drive/MyDriv

In [ ]:
!python detect.py --weights /content/drive/MyDrive/YOLOhelmet/models/YOLO5m.pt --img 640 --conf 0.4 --source '/content/drive/MyDrive/YOLOhelmet/images/Test/scooter (1).jpg'
# display.Image(filename='runs/detect/exp', width=600)

detect: weights=['/content/drive/MyDrive/YOLOhelmet/models/YOLO5m.pt'], source=/content/drive/MyDrive/YOLOhelmet/images/Test/scooter (1).jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /usr/local/lib/python3.10/dist-packages/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-167-g5deff14 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
custom_YOLOv5m summary: 290 layers, 20869098 parameters, 0 gradients, 47.9 GFLOPs
image 1/1 /content/drive/MyDrive/YOLOhelmet/images/Test/scooter (1).jpg: 384x640 1 Scooter, 1 Without Helmet, 55.9ms
Speed: 0.8ms pre-process, 55.9ms inference, 222.8ms NMS per im

In [ ]:
!model.info(verbose=True)

In [ ]:
!python export.py --weights yolov5m.pt --include onnx --simplify

In [ ]:
!python export.py --weights /content/drive/MyDrive/YOLOhelmet/models/YOLO5m.pt --include onnx --simplify

export: data=data/coco128.yaml, weights=['/content/drive/MyDrive/YOLOhelmet/models/YOLO5m.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=True, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-167-g5deff14 Python-3.10.11 torch-2.0.0+cu118 CPU

Fusing layers... 
custom_YOLOv5m summary: 290 layers, 20869098 parameters, 0 gradients, 47.9 GFLOPs

PyTorch: starting from /content/drive/MyDrive/YOLOhelmet/models/YOLO5m.pt with output shape (1, 25200, 10) (40.2 MB)
requirements: Ultralytics requirement "onnx>=1.12.0" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 331.2 MB/s eta 0:00:00

requirements: 1 package updated per ['onnx>=1.12.0']
req